In [12]:
import numpy as np
import pandas as pd
from scipy import optimize
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt






In [13]:
df = pd.read_csv('/content/data.csv', encoding='latin1', engine='python')

print("Available columns:", df.columns.tolist())

no2 = df['no2'].dropna().values.astype(float)

print(f"NO2 samples: {len(no2)}, mean: {np.mean(no2):.2f}")
df.head(10)


Available columns: ['stn_code', 'sampling_date', 'state', 'location', 'agency', 'type', 'so2', 'no2', 'rspm', 'spm', 'location_monitoring_station', 'pm2_5', 'date']
NO2 samples: 419509, mean: 25.81


,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date
0,150,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,NaN,1990-02-01
1,151,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,3.1,7.0,NaN,NaN,NaN,NaN,1990-02-01
2,152,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.2,28.5,NaN,NaN,NaN,NaN,1990-02-01
3,150,March - M031990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.3,14.7,NaN,NaN,NaN,NaN,1990-03-01
4,151,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.7,7.5,NaN,NaN,NaN,NaN,1990-03-01
5,152,March - M031990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.4,25.7,NaN,NaN,NaN,NaN,1990-03-01
6,150,April - M041990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",5.4,17.1,NaN,NaN,NaN,NaN,1990-04-01
7,151,April - M041990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.7,8.7,NaN,NaN,NaN,NaN,1990-04-01
8,152,April - M041990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",4.2,23.0,NaN,NaN,NaN,NaN,1990-04-01
9,151,May - M051990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.0,8.9,NaN,NaN,NaN,NaN,1990-05-01


In [14]:
r = 102316020
ar = 0.05 * (r % 7)
br = 0.3 * ((r % 5) + 1)
print(f"ar = {ar}, br = {br}")

def tr(x):
    return x + ar * np.sin(br * x)


z = tr(no2)
print(f"Valid Z samples: {len(z)}, mean: {np.mean(z):.2f}, std: {np.std(z):.2f}")

ar = 0.1, br = 0.3
Valid Z samples: 419509, mean: 25.81, std: 18.50


In [15]:
def neg_log_lik(params, z_data):
    lam, mu = params
    if lam <= 0:
        return 1e12

    log_c = 0.5 * (np.log(lam) - np.log(np.pi))

    log_pdf = log_c - lam * (z_data - mu)**2
    log_pdf = np.clip(log_pdf, -50, None)
    return -np.sum(log_pdf)



In [16]:
z_mean = np.mean(z)
z_var = np.var(z)
lam0 = 0.5 / z_var

init = np.array([lam0, z_mean])

result = optimize.minimize(neg_log_lik, init, args=(z,), bounds=[(1e-6, None), (None, None)],
                           method='L-BFGS-B')

lam, mu = result.x

In [17]:
c = np.sqrt(lam / np.pi)

print("\nFitted Parameters:")
print(f"λ = {lam:.8f}")
print(f"μ = {mu:.8f}")
print(f"c = {c:.8f}")

norm = c * np.sqrt(np.pi / lam)
print(f"Norm integral: {norm:.6f}")

zz = np.linspace(z.min(), z.max(), 1000)
kde = gaussian_kde(z)
p_data = kde(zz)
p_fit = c * np.exp(-lam * (zz - mu)**2)




Fitted Parameters:
λ = 0.00161365
μ = 25.80619729
c = 0.02266365
Norm integral: 1.000000
